# Calcul des horraires de PM

Description

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

## Get data from website www.cabaigne.net

Run the bash script

### Get and prepare data

In [ ]:
data_dir = "/home/hke/Desktop/projects/boat/when-to-sail/data/"

file_path = data_dir + "horraires-pm-2025.html"
with open(file_path) as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [ ]:
import re

data = []

pattern = r"(marée basse|marée haute)(\d{2}:\d{2})([\d.]+m)"
for el in soup.find_all("table", {"class": "table table-bordered"}):
    date = el.parent.h3.text
    row = {
        "date": date,
        "h PM": [],
        "h BM": [],
        "hde PM": [],
        "hde BM": [],
        }
    for child in el.tbody.children:
        # Effectuer la correspondance
        match = re.match(pattern, child.text)

        # Vérifier si la correspondance a réussi
        if match:
            maree = match.group(1)
            heure = match.group(2)
            hauteur = match.group(3)
            if maree == "marée basse":
                row["h BM"].append(heure)
                row["hde BM"].append(hauteur)
            if maree == "marée haute":
                row["h PM"].append(heure)
                row["hde PM"].append(hauteur)
    data.append(row)

In [ ]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

df = pd.DataFrame(data)
df["datetime"] = df["date"].apply(lambda x: datetime.strptime(x, "%A %d %B %Y"))
df

In [ ]:
df.to_csv(data_dir + "marees-tailles-horraires-2025.csv")